# 1. BPE

In [19]:
# 단어 dictionary 만들기
ori_dictionary = {
    'low': 5,
    'lower': 7,
    'newest': 6,
    'widest': 3
}
ori_dictionary


{'low': 5, 'lower': 7, 'newest': 6, 'widest': 3}

# 데이터 전처리
# -> 단어들을 글자(chracter) 단위로 분리
##### low    -> l o w
##### lower  -> l o w e r
##### newest -> n e w e s t
##### widest -> w i d e s t

In [20]:
dictionary = {}

for words, count in ori_dictionary.items():
  new_word = ''
  for i in words:
    new_word += i + ' '
  dictionary[new_word + '-'] = count
dictionary

{'l o w -': 5, 'l o w e r -': 7, 'n e w e s t -': 6, 'w i d e s t -': 3}

# 1회차
### 분리된 글자를 쌍으로 조합시 횟수 확인

In [21]:
import collections
import re

In [22]:
# defaultdict(int): 초기값 0으로 설정
pairs = collections.defaultdict(int)

for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)-1):
     pairs[symbols[i],symbols[i+1]] += count
print(pairs)

defaultdict(<class 'int'>, {('l', 'o'): 12, ('o', 'w'): 12, ('w', '-'): 5, ('w', 'e'): 13, ('e', 'r'): 7, ('r', '-'): 7, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '-'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3})


In [23]:
# vocabulary 만들기
vocabulary = set()

for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)):
    if symbols[i] == '-':
        pass
    else:
        vocabulary.add(symbols[i])

vocabulary

{'d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w'}

### 빈도수 높은 횟수를 조합 확인

In [24]:
# 횟수 가 높은 세트 확인
best = max(pairs, key=pairs.get)
best

('w', 'e')

### 조합을 dictionary에 업데이트

In [25]:
new_set = ' '.join(best)
new_set

'w e'

In [26]:
new_set.replace(" ", "")

'we'

In [27]:
new_dictionary = {}

for word, count in dictionary.items():
  if new_set in word:
    new_word = word.replace(new_set, new_set.replace(" ", ""))
    new_dictionary[new_word] = dictionary[word]
  else:
    new_dictionary[word] = count
new_dictionary

{'l o w -': 5, 'l o we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}

In [28]:
# l o w e r -> w e -> l o we r
# 5

# 2회차
# 1회차랑 똑같이 진행

In [29]:
dictionary = new_dictionary

pairs = collections.defaultdict(int)

for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)-1):
     pairs[symbols[i],symbols[i+1]] += count
print(pairs)

defaultdict(<class 'int'>, {('l', 'o'): 12, ('o', 'w'): 5, ('w', '-'): 5, ('o', 'we'): 7, ('we', 'r'): 7, ('r', '-'): 7, ('n', 'e'): 6, ('e', 'we'): 6, ('we', 's'): 6, ('s', 't'): 9, ('t', '-'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3, ('e', 's'): 3})


In [30]:
for words, count in dictionary.items():
  symbols = words.split()
  for i in range(len(symbols)):
    if symbols[i] == '-':
        pass
    else:
        vocabulary.add(symbols[i])

vocabulary

{'d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', 'we'}

In [31]:
best = max(pairs, key=pairs.get)
best

('l', 'o')

In [32]:
new_set = ' '.join(best)
new_set

'l o'

In [33]:
new_dictionary = {}

for word, count in dictionary.items():
  if new_set in word:
    new_word = word.replace(new_set, new_set.replace(" ", ""))
    new_dictionary[new_word] = dictionary[word]
  else:
    new_dictionary[word] = count
new_dictionary

{'lo w -': 5, 'lo we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}

# 위 순서대로 원하는 만큼 돌리고 dictionary 및 vocabulary 상시 업데이트

# BPE 알고리즘


In [34]:
import re
import collections

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def make_vocabulary(dictionary):
  global vocabulary
  for words, count in dictionary.items():
    symbols = words.split()
    for i in range(len(symbols)):
        if symbols[i] == '-':
            pass
        else:
            vocabulary.add(symbols[i])

  return vocabulary

def merge_dictionary(pair, dictionary):
    new_dictionary = {}

    for word, count in dictionary.items():
        if pair in word:
            new_word = word.replace(pair, pair.replace(" ", ""))
            new_dictionary[new_word] = dictionary[word]
        else:
            new_dictionary[word] = count
    return new_dictionary

In [35]:
dictionary = {'l o w -': 5,
         'l o w e r -': 7,
         'n e w e s t -':6,
         'w i d e s t -':3
         }

vocabulary = set()
num_merges = 10

for i in range(num_merges):
    vocabulary = make_vocabulary(dictionary)
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    new_set = ' '.join(best)
    dictionary = merge_dictionary(new_set, dictionary)

    print("new merge: {}".format(best))
    print('dictionary: {}'.format(dictionary))
    print('vocabulary: {}'.format(vocabulary))
    print('=' * 300)

new merge: ('w', 'e')
dictionary: {'l o w -': 5, 'l o we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}
vocabulary: {'t', 'l', 'w', 'o', 'r', 'd', 'i', 'e', 'n', 's'}
new merge: ('l', 'o')
dictionary: {'lo w -': 5, 'lo we r -': 7, 'n e we s t -': 6, 'w i d e s t -': 3}
vocabulary: {'t', 'l', 'w', 'o', 'r', 'd', 'i', 'e', 'n', 'we', 's'}
new merge: ('s', 't')
dictionary: {'lo w -': 5, 'lo we r -': 7, 'n e we st -': 6, 'w i d e st -': 3}
vocabulary: {'t', 'l', 'w', 'o', 'r', 'd', 'i', 'e', 'lo', 'n', 'we', 's'}
new merge: ('st', '-')
dictionary: {'lo w -': 5, 'lo we r -': 7, 'n e we st-': 6, 'w i d e st-': 3}
vocabulary: {'t', 'l', 'st', 'w', 'o', 'r', 'd', 'i', 'e', 'lo', 'n', 'we', 's'}
new merge: ('lo', 'we')
dictionary: {'lo w -': 5, 'lowe r -': 7, 'n e we st-': 6, 'w i d e st-': 3}
vocabulary: {'t', 'l', 'st', 'w', 'o', 'r', 'st-', 'd', 'i', 'e', 'lo', 'n', 'we', 's'}
new merge: ('lowe', 'r')
dictionary: {'lo w -': 5, 'lower -': 7, 'n e we st-': 6, 'w i d e st-': 3}
vocabulary: {'t

In [43]:
# 한국어 예제

corpus = [
    "오늘 점심에 배가 너무 고파서 밥을 너무 많이 먹었다.",
    "오늘 점심에 배가 고파서 밥을 많이 먹었다.",
    "오늘 배가 너무 고파서 점심에 밥을 너무 많이 먹었다.",
    "오늘 점심에 배가 고파서 버스를 많이 먹었다.",
    "펩시는 사랑하지 않아",
    "어제 저녁에 밥을 너무 많이 먹었더니 배가 부르다.",
    "이따가 오후 7시에 출발하는 비행기가 3시간 연착 되었다고 하네요."
]


In [44]:
from typing import Dict, List, Tuple
import re
from collections import defaultdict

# 불용어 제거
SPECIALS = "".join([".", ",", ";" ,":", "!","?", "'", '"', " " ])

def preprocess(text: str, only_kor: bool=True):
  """ 한국어 문장을 옵션에 맞게 전처리 """
  # 한국어 모음과 특수 문자, 숫자 및 영어 제거
  if only_kor:
    text = re.sub(f"[^가-힣| |]+", "", text)
  else:
    text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|{SPECIALS}|a-zA-Z]+", "", text)

  # 연속 공백 제거
  text = re.sub(" +", "", text)

  # 좌우 불필요한 공백 제거
  return text.strip()

In [45]:
def pseudo_get_vocab(corpus: List[str]) -> Dict[str, int]:
  """더미 데이터를 읽어와 단어 사전 구축"""
  vocab = defaultdict(int)
  for line in corpus:
    tokens = preprocess(line).strip().split(" ")
    for token in tokens:
      vocab[" ".join(list(token)) + " </w>"] += 1
  return dict(vocab)

In [46]:
kor_dic = pseudo_get_vocab(corpus)
kor_dic

{'오 늘 점 심 에 배 가 너 무 고 파 서 밥 을 너 무 많 이 먹 었 다 </w>': 1,
 '오 늘 점 심 에 배 가 고 파 서 밥 을 많 이 먹 었 다 </w>': 1,
 '오 늘 배 가 너 무 고 파 서 점 심 에 밥 을 너 무 많 이 먹 었 다 </w>': 1,
 '오 늘 점 심 에 배 가 고 파 서 버 스 를 많 이 먹 었 다 </w>': 1,
 '펩 시 는 사 랑 하 지 않 아 </w>': 1,
 '어 제 저 녁 에 밥 을 너 무 많 이 먹 었 더 니 배 가 부 르 다 </w>': 1,
 '이 따 가 오 후 시 에 출 발 하 는 비 행 기 가 시 간 연 착 되 었 다 고 하 네 요 </w>': 1}

In [47]:
vocabulary = set()
num_merges = 20

for i in range(num_merges):
    vocabulary = make_vocabulary(kor_dic)
    pairs = get_stats(kor_dic)
    best = max(pairs, key=pairs.get)
    new_set = ' '.join(best)
    kor_dic = merge_dictionary(new_set, kor_dic)

    print("new merge: {}".format(best))
    print('dictionary: {}'.format(kor_dic))
    print('vocabulary: {}'.format(vocabulary))
    print('=' * 300)

new merge: ('배', '가')
dictionary: {'오 늘 점 심 에 배가 너 무 고 파 서 밥 을 너 무 많 이 먹 었 다 </w>': 1, '오 늘 점 심 에 배가 고 파 서 밥 을 많 이 먹 었 다 </w>': 1, '오 늘 배가 너 무 고 파 서 점 심 에 밥 을 너 무 많 이 먹 었 다 </w>': 1, '오 늘 점 심 에 배가 고 파 서 버 스 를 많 이 먹 었 다 </w>': 1, '펩 시 는 사 랑 하 지 않 아 </w>': 1, '어 제 저 녁 에 밥 을 너 무 많 이 먹 었 더 니 배가 부 르 다 </w>': 1, '이 따 가 오 후 시 에 출 발 하 는 비 행 기 가 시 간 연 착 되 었 다 고 하 네 요 </w>': 1}
vocabulary: {'르', '출', '요', '가', '무', '</w>', '지', '에', '서', '너', '버', '을', '연', '늘', '사', '심', '오', '시', '많', '먹', '니', '점', '비', '펩', '랑', '따', '행', '기', '아', '되', '다', '후', '발', '었', '간', '를', '않', '이', '착', '더', '고', '어', '부', '밥', '는', '하', '네', '저', '녁', '파', '스', '배', '제'}
new merge: ('너', '무')
dictionary: {'오 늘 점 심 에 배가 너무 고 파 서 밥 을 너무 많 이 먹 었 다 </w>': 1, '오 늘 점 심 에 배가 고 파 서 밥 을 많 이 먹 었 다 </w>': 1, '오 늘 배가 너무 고 파 서 점 심 에 밥 을 너무 많 이 먹 었 다 </w>': 1, '오 늘 점 심 에 배가 고 파 서 버 스 를 많 이 먹 었 다 </w>': 1, '펩 시 는 사 랑 하 지 않 아 </w>': 1, '어 제 저 녁 에 밥 을 너무 많 이 먹 었 더 니 배가 부 르 다 </w>': 1, '이 따 가 오 후 시 에 출 발 하 는 비 행 기 가 시 간 연 착 되 었 다 